# Import Libraries

In [ ]:
import os
import pandas as pd
import sqlparse

# Define Paths

In [ ]:
# Directory of Repo where the SQL DDL will be saved
repo_dir = input("Enter Repo directory path, to save the repo files:")
table_metadata_filepath = input("Enter Table Metadata file path:")
object_definitions_filepath = input("Enter Object Definitions file path:")

# Define Functions

## Table DDL Parsing

### generate_create_table_script

In [ ]:
# Function to generate the CREATE TABLE script
def generate_create_table_script(df):
    # Initialise the varchar_max_check:
        # This will help with creating the DDL where a varchar(max) has been found in the SQL
    varchar_max_check = False

    # Get the Schema and Table name for the table being parsed
    schema_name = df['SchemaName'].iloc[0]
    table_name = df['TableName'].iloc[0]
    
    # Initialise the CREATE TABLE Script, with the Schema and Table name
    create_table_script = f"CREATE TABLE [{schema_name}].[{table_name}]\n(\n"
    
    # Iterate through the rows in the dataframe
    for _, row in df.iterrows():
        ## Columns and Data Types ##
        column_definition = f"\t[{row['ColumnName']}] [{row['DataType']}]"
        
        ### Strings ###
        if row['DataType'] in ['varchar', 'nvarchar', 'varbinary', 'char']:

            # Look for varchar(max) fields
            if row['MaxLength'] == -1:
                varchar_max_check = True
                column_definition += "(MAX)"
            else:
                column_definition += f"({row['MaxLength']})"

        ### Numbers / Decimal ###
        elif row['DataType'] in ['decimal', 'numeric']:
            column_definition += f"({row['precision']},{row['scale']})"

        ### Dates (datetime2) ###
        elif row['DataType'] == 'datetime2':
            column_definition += f"({row['scale']})"
        

        ## Determine identity columns ##
        if row['is_identity']:
            column_definition += " IDENTITY(1,1)"
        

        ## Determine NULL / NOT NULL columns ##
        if row['is_nullable']:
            column_definition += " NULL"
        else:
            column_definition += " NOT NULL"
        

        ## For each column definition enter a new line ##
        create_table_script += column_definition + ",\n"
    

    ## Strip any additional new line characters ##
    create_table_script = create_table_script.rstrip(",\n") + "\n)\n"
    

    ## Table Distribution Settings ##
    distribution_type = df["DistributionType"].iloc[0]
    distribution_col = df["HashDistributionColumnName"].iloc[0]

    ### Obtain Hash Distribution Columns ###
    if distribution_type == 'HASH':
        if distribution_col:
            distribution_type += f"([{distribution_col}])"
        else:
            raise ValueError(f"No column provided for Hash distributed table: [{schema_name}].[{table_name}]!")


    ## Table Indexing Settings ##
    index_type = df["IndexType"].iloc[0]


    ## Add Table Distribution and Index Settings to the CREATE TABLE Script ##
    if varchar_max_check:
        create_table_script += f"WITH\n(\n\tDISTRIBUTION = {distribution_type},\n\tHEAP\n)" # If varchar(max) detected then use HEAP Index
    else:
        if index_type:
            create_table_script += f"WITH\n(\n\tDISTRIBUTION = {distribution_type},\n\t{index_type} INDEX\n)"
        else:
            create_table_script += f"WITH\n(\n\tDISTRIBUTION = {distribution_type},\n\tHEAP\n)" # If no index setting found then use HEAD Index as default
    
    return create_table_script

### parse_table_metadata

In [ ]:
def parse_table_metadata(repo_dir, df_metadata):
    # Split the dataframe by each table in the TABLE_NAME field
    tables = df_metadata.groupby(['SchemaName', 'TableName'])

    table_count = 0

    # Iterate over each table and generate the CREATE TABLE script
    for (schema_name, table_name), table_df in tables:
        # Generate the CREATE TABLE script
        create_table_script = generate_create_table_script(table_df)
        
        # Create the sub-folder named after the schema for the table
        folder_path = os.path.join(repo_dir, schema_name, "Tables")
        os.makedirs(folder_path, exist_ok=True)
        
        # Save the generated CREATE TABLE SQL script as a .sql file
        file_path = os.path.join(folder_path, f"{table_name}.sql")
        with open(file_path, 'w') as file:
            file.write(create_table_script)

        table_count += 1

    print(f"""
        CREATE TABLE scripts have been generated and saved successfully.
        {table_count} table DDL files processed
        """
        )

## Object DDL Parsing

### parse_object_definitions

In [ ]:
def parse_object_definitions(repo_dir, df_definitions):
    # Group the dataframe by 'ObjectType'
    obj_grouped = df_definitions.groupby('ObjectType')

    # Run through each object type
    for object_type, objects_df in obj_grouped:
        object_count = 0

        # Run through each object in the group
        for _, row in objects_df.iterrows():
            schema_name = row['SchemaName']
            object_name = row['ObjectName']
            object_definition = row['ObjectDefinition']
            
            formatted_view = sqlparse.format(object_definition, reindent=False, keyword_case='upper')
            
            # Create the sub-folder named after the schema for the view
            folder_path = os.path.join(repo_dir, schema_name, object_type)
            os.makedirs(folder_path, exist_ok=True)
            
            # Save the generated CREATE VIEW SQL script as a .sql file
            file_path = os.path.join(folder_path, f"{object_name}.sql")
            with open(file_path, 'w') as file:
                file.write(formatted_view)

            object_count += 1

        print(f"""
            CREATE scripts for {str(object_type).upper()} have been generated and saved successfully.
            {object_count} DDL files processed
            """
            )

# Import SQL Metadata / Definitions

In [ ]:
# Import Table Metadata
table_metadata = pd.read_json(fr"{table_metadata_filepath}")

# Display Table Metadata
table_metadata

In [ ]:
# Import Object Definitions
object_definitions = pd.read_json(fr"{object_definitions_filepath}")

# Display Object Definitions
object_definitions

# Parse Metadata / Definitions to Repo Folders

In [ ]:
parse_table_metadata(fr"{repo_dir}", table_metadata)
parse_object_definitions(fr"{repo_dir}", object_definitions)